In [1]:
import tensorflow as tf
import numpy as np
import nltk
from nltk.corpus import gutenberg

# word embedding
词向量模型，主要功能是用于表示

参考
- cs224n 第二讲和第三讲
- speech and language processing chapter 16 -- semantic with dense vectors
- https://www.jiqizhixin.com/articles/2020-06-30-11 推荐系统 embedding 技术实践总结
- 刘建平 word2vec原理(三) 基于Negative Sampling的模型

# 文本表征的方法
文本表征的目的是将文本转化成机器学习算法可以处理的数字形式，换句话说就是将文本使用数学语言进行表示。
- 词袋模型 one-hot\tf-idf
- 主题模型，LSA\PLSA\LDA等
- 词向量模型 word2vec fastText glove
- 动态词向量模型 elmo BERT GPT

词向量模型将一个词表示成一个先对要短（1000以内的长度）以及稠密的向量（大部分值不为0）
词袋模型简单，但是存在维数灾难和语义鸿沟的问题
主题模型训练计算复杂度高
早期的词向量的研究源于语言模型，比附NNLM和RNNLM, 词向量是副产物。
**语言模型，就是用来计算一个词序列的概率的模型。**


分布式假设：相同上下文语境的词具有相似含义，并且由此引出了word2vec和fastText等。这类词向量中，虽然其本质仍然是语言模型，但其目标是生成词向量。

embdding技术尽管非常实用，但是依然存在一些问题。【3】比如
- 增量更新前后语义不变
- cover多个维度的特征
- 多模态向量融合
- 长尾数据资料少难以充分训练
- Embedding空间分布，影响模型泛化误差

fastText 相比于word2vec, 训练词向量会考虑subword。还可以进行文本分类

glove 的特点
glove相比于LSA， 采用Adagrad 对最小平方损失进行优化，可以算是一种经过优化的矩阵分解算法。

glove 相比于word2vec， 使用了全局语料，同时glove不能进行在线学习，因为其需要统计固定语料信息

glove的损失韩式最小平方损失函数，相比于word2vec的带权重的交叉熵，


elmo、GPT和bert 
这三个均为动态词向量。动态词向量的特点是在不同上下文环境下，词向量也会不相同。这就解决了一词多义问题。

elmo使用LSTM进行提取特征，使用双向语言模型
GPT和bert使用transformer来提取特征
bert使用双向语言模型

word2vec 有两种实现方式, CBOW 和 Skip-gram

CBOW 使用上下文背景词来预测目标词
Skip-gram使用目标词来预测背景词


为了加快word2vec的训练速度，使用了negative sampling 和 Hierarchical softmax.



如果使用softmax 那么输出时间复杂度为O(N). 如果词的数量越多，那么就会花费更长时间来做输出。同时有这么多的输出，其概率分布很可能会比较均衡，难以导致模型的收敛。
hierarchical softmax 的核心想法，是将输出形式变为二叉树的形式。

negative sampling 使用采样的方法。对于一个训练样本，存在一个中心词w以及上下背景词2c个， 记为context(w)。中心词w和背景词具有相关关系，可以认定其为正例，通过negative sampling采样，我们可以得到neg个和w不同的中心词wi-neg. 这样context(w)和w_i组成了neg个并不真实存在负例。我们基于一个正例和neg个负例，来进行二元逻辑回归。最后得到负采样模型参数以及每个词的词向量。

存在两个问题
1）如何进行负采样？
2）如何通过一个正例和neg个负例进行二元逻辑回归。

如何进行负采样
首先我们可以收集到所有的词汇，并统计每个词汇的词频。这样我们就可以得到每个词汇的长度，其长度和词汇的词频有关。词频越高，则长度越长。



在word2vec 中，分子和分母都取3/4次幂

在采样前，将长度为1的现端划分为M等分，这里M>>V。这样可以保证每个词对应的线段都会划分成对应的小块。而M份的
每一份都会落在某一个词对应的线段商。在采样的时候
只需要从M个位置中采样出neg个位置就行。

在word2vec中，M取值默认为10^8


In [2]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [16]:
data = gutenberg.words("austen-emma.txt")[:1000]

In [4]:
len(data)

192427

In [17]:
word_count = dict()
for d in data:
    d = d.lower()
    word_count[d] = word_count.get(d, 0) + 1

In [18]:
word_list = [(k, v) for k, v in word_count.items()]
word_list.sort(key=lambda x: x[1])

In [19]:
TOP = 10
WINDOW = 2

In [20]:
# 筛选词
word2id = {"UNK": 0}

top_num = len(word_list)*10//100
stop_list = word_list[:top_num] + word_list[-top_num:]
stop_set = set([k for k, _ in stop_list])
word2id =  {k:i+1 for i, (k, _) in  enumerate(word_list[top_num:-top_num])}
word2id["UNK"] = 0

In [21]:
VOCAB_SIZE = len(word2id)
EMBED_SIZE = 10

In [22]:
data_len

192427

# 数据预处理
将文本中的词进行转换，转换成数字

In [89]:
data_len = len(data)
context_data = []
label_data = []
temp_windows = ["P"*WINDOW]
for i, d in enumerate(data):
    temp_windows.append(d)
    
    context = []
    if len(temp_windows) == 2*WINDOW+1:
        for j, t in enumerate(temp_windows):
            if t == "P":
                continue
            if j == WINDOW:
                continue
            context.append(word2id.get(t, 0))
        label.append(word2id.get(temp_windows[WINDOW], 0))
        temp_windows.pop(0)
    
    if context:
        context_data.append(context)
        label_data.append(label[0])
            
    if i == data_len-1:
        context = []
        for j in range(WINDOW):
            
            
            for k, t in enumerate(temp_windows[j:]):
                if k == WINDOW:
                    continue
                context.append(word2id.get(t, 0))
            
            label_data.append(word2id.get(temp_windows[j+WINDOW], 0))
            
            context_data.append(context)
#             label_data.append(label)
            

# 使用one-hot编码

In [90]:
context_data = [tf.keras.utils.to_categorical(d).sum(axis=0) for d in context_data]

In [81]:
VOCAB_SIZE

321

In [95]:
context_data = tf.keras.preprocessing.sequence.pad_sequences(context_data, maxlen=VOCAB_SIZE, padding="post")

In [96]:
context_data = context_data.astype('float64')
label_data = np.array(label_data).astype("float64")

In [94]:
type(context_data)

list

In [43]:
# 构造训练专用数据集

In [97]:
dataset = tf.data.Dataset.from_tensor_slices((context_data, label_data))

In [98]:
dataset = dataset.shuffle(data_len).batch(100, drop_remainder=True)

In [105]:

class CBOW(tf.keras.Model):
    
    def __init__(self, vocab_size, embed_size):
        super(CBOW, self).__init__()
        
        self.embed = tf.keras.layers.Dense(embed_size)
        self.out = tf.keras.layers.Dense(vocab_size, activation="softmax")
        
    
    def call(self, input_data):
        
        x = self.embed(input_data)
        logits = self.out(x)
        
        return logits

In [106]:
model = CBOW(VOCAB_SIZE, EMBED_SIZE)

In [37]:
result = model(tf.constant([[0, 1, 0]]))

In [38]:
result.shape

TensorShape([1, 321])

In [32]:
optimiser = tf.keras.optimizers.Adam()
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [109]:
@tf.function()
def train_step(input_x, input_target):
    
    with tf.GradientTape() as tape:
        logit = model(input_x)
        loss = loss_func(input_target, logit)
    
    variables = model.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimiser.apply_gradients(zip(gradients, variables))
    
    return loss

In [102]:
train_x.shape

TensorShape([100, 321])

In [107]:
logit = model(train_x)
# loss = loss_func(input_target, logit)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [110]:
EPOCH = 100

for i in range(EPOCH):
    
    for k, (train_x, train_y) in enumerate(dataset):
        
        loss = train_step(train_x, train_y)
        
        if k % 100 == 0:
            print("epoch {0} batch {1} loss value is {2}".format(i, k, loss))

epoch 0 batch 0 loss value is 5.771358013153076
epoch 1 batch 0 loss value is 5.771195888519287
epoch 2 batch 0 loss value is 5.770995616912842
epoch 3 batch 0 loss value is 5.770693302154541
epoch 4 batch 0 loss value is 5.77040958404541
epoch 5 batch 0 loss value is 5.7698893547058105
epoch 6 batch 0 loss value is 5.769169807434082
epoch 7 batch 0 loss value is 5.7680253982543945
epoch 8 batch 0 loss value is 5.766364097595215
epoch 9 batch 0 loss value is 5.763433456420898
epoch 10 batch 0 loss value is 5.758155345916748
epoch 11 batch 0 loss value is 5.748624801635742
epoch 12 batch 0 loss value is 5.725463390350342
epoch 13 batch 0 loss value is 5.681973934173584
epoch 14 batch 0 loss value is 5.597226619720459
epoch 15 batch 0 loss value is 5.429547309875488
epoch 16 batch 0 loss value is 5.226442337036133
epoch 17 batch 0 loss value is 5.0365986824035645
epoch 18 batch 0 loss value is 4.912534236907959
epoch 19 batch 0 loss value is 4.852176666259766
epoch 20 batch 0 loss value 